In [ ]:
import sys
sys.path.append('/home/jovyan/work/')
sys.path.append('/home/jovyan/work/src')
sys.path.append('/home/jovyan/work/src/data')
sys.path.append('/home/jovyan/work/src/data_explore')
sys.path.append('/home/jovyan/work/src/features')
sys.path.append('/home/jovyan/work/src/models')
sys.path.append('/home/jovyan/work/src/analysis')


In [ ]:
print(sys.path)

In [ ]:
from src import data,data_explore,features
from data import create_input_data
from data_explore import explore_util
from features import build_features
from models import train_model,predict_model
from analysis import analyze_pred

In [ ]:
#df = create_input_data.read_data(csv_file_name="../data/processed/past_25_completed_qm.csv", parse_dates_col=[4,5,6])
df = create_input_data.read_data(csv_file_name="../data/processed/correct_anon_past_25_completed_qm.csv", parse_dates_col=[4,5,6])
df

In [ ]:
num_zeros, non_zeros = explore_util.get_num_of_zeros(df,'qm25')


In [ ]:
explore_util.explore_five_plots(df=df,col_name_list=['qm1','qm2','qm12','qm23','qm24'], num_col=5, color_list=['g','r','b','c','m'],lo=300, hi=400)

In [ ]:
df = build_features.remove_default_cols(df)
df

In [ ]:
df = build_features.slice_df(df,1000,323216)
df

In [ ]:
### Remove the queue_minutes column from the training set
X_historydata=build_features.remove_specific_col(df,'queue_minutes')
Y_waittimedata=df['queue_minutes']

In [ ]:
X_historydata
#Y_waittimedata

In [ ]:
X_historydata_train_rm, X_historydata_test_rm, Y_waittimedata_train, Y_waittimedata_test=build_features.split_training_test_data(X_historydata, Y_waittimedata)

In [ ]:
X_historydata_test = build_features.remove_specific_col(X_historydata_test_rm,'run_minutes')
X_historydata_train = build_features.remove_specific_col(X_historydata_train_rm,'run_minutes')

In [ ]:
X_historydata_train

In [ ]:
X_historydata_norm, scaler = build_features.standardization(X_historydata_train)

In [ ]:
#h5_file="../models/25past_corrected_qm_model.h5"
model = train_model.create_model_with_asymmetric_loss_2(input_shape=32)
#checkpoint_cb,early_stopping_cb = train_model.create_checkpoint_cb(h5_file=h5_file)
#model, hist = train_model.train_model(X_historydata_norm, Y_waittimedata_train, model, epochs=150, batch_size=64, validation_spilt=0.2)
early_stopping_cb = train_model.set_early_stopping()
model, hist = train_model.train_model_cb(X_historydata_norm, Y_waittimedata_train, model, early_stopping_cb,epochs=150, batch_size=64, validation_spilt=0.2)

In [ ]:
predict_model.evaluate(model,scaler,X_historydata_test, Y_waittimedata_test)

In [ ]:
from importlib import reload
reload(predict_model)
y_pred = predict_model.predict_and_compare(model,scaler, X_historydata_test, Y_waittimedata_test)

In [ ]:
reload(predict_model)
predict_model.compare_plots_pred_vs_true(Y_waittimedata_test, y_pred, 150, 200)

In [ ]:
predict_model.compare_plots_pred_vs_true(Y_waittimedata_test, y_pred, 200,250)


In [ ]:
predict_model.inspect_test_data(X_historydata_test,Y_waittimedata_test,y_pred,207,210, "set 1: 207: 210")
predict_model.inspect_test_data(X_historydata_test,Y_waittimedata_test,y_pred,225,228, "set2: 225-228")
predict_model.inspect_test_data(X_historydata_test,Y_waittimedata_test,y_pred,228, 231, "set3: 228-231")

In [ ]:
print(X_historydata_test_rm)
run_mins = X_historydata_test_rm["run_minutes"]
run_mins = run_mins.to_numpy()
print("run_minutes" + str(run_mins))
nnodes = X_historydata_test["nnodes"]
nnodes = nnodes.to_numpy()
print(" nnodes : " + str(nnodes))


In [ ]:
reload(analyze_pred)
analyze_pred.analysis_pred_vs_orig(2000,200, Y_waittimedata_test, y_pred.ravel(), nnodes,run_mins,
                                   provision_time=20, tolerance_factor=3, print_flag=0)

In [ ]:
reload(analyze_pred)
analyze_pred.analysis_pred_vs_orig(0,80554, Y_waittimedata_test, y_pred.ravel(), nnodes,run_mins,
                                   provision_time=20, tolerance_factor=3, print_flag=0)

In [ ]:
analyze_pred.analysis_pred_vs_orig(0,80554, Y_waittimedata_test, y_pred.ravel(), nnodes,run_mins,
                                   provision_time=45, tolerance_factor=3, print_flag=0)

In [ ]:
analyze_pred.analysis_pred_vs_orig(0,80554, Y_waittimedata_test, y_pred.ravel(), nnodes,run_mins,
                                   provision_time=60, tolerance_factor=3, print_flag=0)

In [ ]:
reload(analyze_pred)
analyze_pred.plot_pred_vs_orig(2000,50,Y_waittimedata_test,y_pred.ravel())

In [ ]:
reload(analyze_pred)
from sklearn.metrics import r2_score,mean_absolute_error


def analysis_pred(provision_time):
    ptime = []
    num_scheduled=[] 
    num_missed=[]
    not_missed=[]
    wrongly_scheduled=[]
    total_num=[]
    past_K=[]
    r2_scores=[]
    maes = []
    provision_time, n_scheduled, n_missed, no_missed, wrong_scheduled, t_num = analyze_pred.analysis_pred(lo,offset, Y_waittimedata_test, y_pred.ravel(), nnodes,run_mins,
                               provision_time=provision_time, tolerance_factor=tolerance_factor, print_flag=0)
    past_K.append("25")
    ptime.append(provision_time)
    num_scheduled.append(n_scheduled)
    num_missed.append(n_missed)
    not_missed.append(no_missed)
    wrongly_scheduled.append(wrong_scheduled)
    total_num.append(t_num)
    r2s=r2_score(Y_waittimedata_test.to_numpy(),y_pred)
    r2_scores.append(r2s)     
    maes.append(mean_absolute_error(Y_waittimedata_test.to_numpy(),y_pred))
    return past_K,ptime, num_scheduled, num_missed, not_missed, wrongly_scheduled, total_num,r2_scores,maes
    

In [ ]:
lo=0
offset=80554
provision_times=[20, 45, 60, 90]
tolerance_factor=3
ptime = []
num_scheduled=[] 
num_missed=[]
not_missed=[]
wrongly_scheduled=[]
total_num=[]
past_K=[]
print("asymetric loss function: 2 times penality for over-prediction")
for provision_time in provision_times:
    print(" ========= Provision time = " + str(provision_time))
    pastk, ptime, num_scheduled, num_missed, not_missed, wrongly_scheduled, total_num, r2_scores, maes = analysis_pred(provision_time)
    rdf = analyze_pred.add_to_df(pastk, ptime, num_scheduled, num_missed, not_missed, wrongly_scheduled, total_num,r2_scores,maes)
    print(rdf.to_string())

In [ ]:
analyze_pred.plot_pred_vs_orig(5000,50,Y_waittimedata_test,y_pred.ravel())

In [ ]:
from sklearn.metrics import r2_score
r2_score(Y_waittimedata_test.to_numpy(),y_pred)